# TS Processing

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from importlib import reload

In [2]:
import TimeSeriesGenerator as TSG
import TimeSeries as TS
import DWMYTransformator as T
import DWMYGroupTransformator as GT
import DWMYDFTransformator as DFT

import TSToIndexTransformator as TST
import PipelineTSToIndex as PI

import TripletSaverAndLoader as TSL

In [3]:
%load_ext autoreload
%autoreload 1

In [4]:
tsg = TSG.TimeSeriesGenerator()
ts = TS.TimeSeries()
t = T.DWMYTransformator()
gt = GT.DWMYGroupTransformator()
dft = DFT.DWMYDFTransformator()
tsl = TSL.TripletSaverAndLoader(None)

## Data Generator

def generate_df(ts_lengths=[20, 30, 40, 50], int_data=True):
    df = pd.DataFrame()
    id_list = []
    data_list = []
    i = 0
    for n in ts_lengths:
        id_list.append(i)
        i = i + 1
        data_list.append(tsg.generate_day_data(int_data=False, n=n))

    df["ID"] = id_list
    df["DATA"] = data_list
    
    return df

In [5]:
df_gen = tsg.generate_sample_ts_df()
df_gen

,ID,TS_DATA
0,10,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."
1,20,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."
2,30,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."
3,40,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."


In [6]:
ts_list = df_gen.iloc[0,1]

In [7]:
tsl.save_triplet(df_gen, "test_df", "ID", "DATA")

--Meantime Duration of Saving Triplet Duration:  is: 0.0 minutes / 0.0 seconds


In [8]:
df_all = tsl.load_triplet("test_df", "ID", "DATA")

--Meantime Duration of Loading Triplet Duration:  is: 0.0 minutes / 0.0 seconds


In [9]:
df_all

,ID,TS_DATA
0,10,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."
1,20,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."
2,30,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."
3,40,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."


In [10]:
df_all.equals(df_gen)

True

## One TS

### Metaparametry

In [11]:
atr_names = ["TIME", "VALUE"]
time_type = "d"
group_fun = sum

### Postupně

#### TS jako list

In [12]:
ts_list = df_all.iloc[0,1]
ts_list[0:2]

[(datetime.datetime(2019, 8, 9, 0, 0), 1),
 (datetime.datetime(2019, 9, 9, 0, 0), 1)]

#### Vytvořím si z toho DF

In [13]:
ts.set_ts_list(ts_list, atr_names)
df = ts.get_ts_df()
df.head(2)

,TIME,VALUE
0,2019-08-01,1
1,2019-08-01,1


#### Transformuji čas

In [14]:
df = dft.fit(df, time_type, atr_names)
df.head()

,VALUE,TIME_d
0,1,20190801
1,1,20190801
2,1,20190805
3,1,20190809
4,1,20190813


#### Grouping

In [15]:
df_out = gt.fit(df, time_type, atr_names, group_fun)
df_out.head()

,TIME_d,VALUE
0,20190801,2
1,20190805,1
2,20190809,1
3,20190813,2
4,20190822,1


#### Do indexu

In [16]:
reload(TST)
tst = TST.TSToIndexTransformator()

In [17]:
a = tst.fitpredict(df_out["VALUE"].array)
a

0.6643574057122259

### Pipeline

In [18]:
reload(PI)
p = PI.PipelineTSToIndex()

In [19]:
print(p._create_index_for_ts_list(ts_list, time_type, atr_names, group_fun))

0.6643574057122259


## TS DF

In [20]:
df_all

,ID,TS_DATA
0,10,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."
1,20,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."
2,30,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."
3,40,"[(2019-08-09 00:00:00, 1), (2019-09-09 00:00:0..."


In [21]:
reload(PI)
p = PI.PipelineTSToIndex()

In [22]:
atr_names_df = ["ID", "TS_DATA"]
atr_names_ts = ["TIME", "VALUE"]
file_name = "test_df"

In [23]:
entropy = p.execute(file_name, time_type, atr_names_df, atr_names_ts, group_fun, fun_type="entropy")

--Meantime Duration of Loading Triplet Duration:  is: 0.0 minutes / 0.01 seconds


In [24]:
entropy

,DATA_ENTROPY,ID
0,0.664357,10
1,0.515273,20
2,0.639550,30
3,0.621226,40
